# Magic Paste





## Описание


### **Концепция**  
Интеллектуальный ассистент для "бесшовной" работы с текстом, расширяющий  
возможности стандартного ввода в любой среде операционной системы.  

### **Проблема**  
При использовании внешних LLM-инструментов (ChatGPT, переводчики, корректоры)   
пользователь сталкивается с постоянным переключением контекста. Процесс  
"выделить - скопировать - переключить окно - вставить - получить ответ -  
скопировать - вернуться - вставить" нарушает рабочий поток и снижает  
концентрацию.

### **Техническое решение**  
Magic Paste интегрирует возможности языковой модели `Gemma-3n E4B Instruct`  
непосредственно в активное приложение пользователя через **плавающий оверлейный**  
**интерфейс**.  
Ключевая особенность реализации - архитектура **без захвата фокуса**: окно  
ассистента визуально доступно, но системный курсор и фокус ввода остаются в  
основном рабочем приложении.

### **Сценарий использования**  
1.  **Активация:** Работая в IDE, браузере или мессенджере, пользователь  
вызывает ассистента голосовой командой или хоткеем. Появляется ненавязчивое  
диалоговое окно поверх интерфейса.  
2.  **Контекст и Ввод:** Пользователь надиктовывает текст голосом *либо*  
ассистент автоматически захватывает уже выделенный текст в активном окне.  
3.  **Обработка:** Пользователь голосом отдает команду на модификацию  
(например: *"Сделай текст более официальным"*, *"Исправь баги в коде"*,  
*"Переведи на английский"*).  
4.  **Результат:** Magic Paste обрабатывает запрос и автоматически вставляет  
готовый результат в позицию курсора активного приложения.  


### **Итог:**
Инструмент устраняет необходимость ручного копирования и переключения  
окон, обеспечивая единый интерфейс взаимодействия с ИИ в любой точке системы.  



## ⚡ Библиотеки, модель, данные, функция оценки

### Установка библиотек

In [ ]:
# ⚠️ Работающий конфиг. Не менять!

%%capture
import os, re
# для torchcodec нужен FFmpeg 7+
!add-apt-repository -y ppa:ubuntuhandbook1/ffmpeg8
!apt-get update && apt-get install -y ffmpeg libavutil-dev libavcodec-dev libavformat-dev

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    import torch; v = re.match(r'[\d]{1,}\.[\d]{1,}', str(torch.__version__)).group(0)
    xformers = 'xformers==' + {'2.10':'0.0.34','2.9':'0.0.33.post1','2.8':'0.0.32.post2'}.get(v, "0.0.34")
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth_zoo bitsandbytes accelerate {xformers} peft trl triton unsloth

!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install  torchcodec==0.9 # нужна версия 0.9 для torch 2.9
import torch; torch._dynamo.config.recompile_limit = 64;

!pip install --no-deps --upgrade timm # Only for Gemma 3N

### Загрузка Gemma-3n

In [ ]:
from unsloth import FastModel
import torch
import time

# загрузка с ограничением памяти
model, processor = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    # возmмем побольше для аудио-токенов в будущем
    # max_seq_length = 512, # было 4096 и 1024
    max_seq_length = 2048, # длинный промпт
    load_in_4bit = True,
    # оставляем 30% памяти GPU свободной для инференса и аудио-эмбеддингов
    gpu_memory_utilization = 0.60,  #
)

# Отключаем кэш (критично для обучения!)
model.config.use_cache = False
model.config.pretraining_tp = 1 # Иногда помогает для Gemma


# настройка токенизатора
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(processor, chat_template="gemma3n")

tokenizer.model_max_length = 512
# tokenizer.padding_side = "right"

print("✅ Модель загружена")

###  Загрузка  данных
- JSON примеры сгенерированы с помощью gemini 3 Flash
- Aудио сгенерировано на основе полученных примеров с помошью edgeTTS
- код генерации приведен в репозитории

In [ ]:

from datasets import load_dataset, Audio
import os

# клонируем репозиторий
if not os.path.exists("project_data"):
    !git clone https://github.com/iamgm/speech-toolformer.git project_data

# unzip аудио
if not os.path.exists("project_data/data/audio_dataset"):
    !unzip -o -q project_data/data/audio.zip -d project_data/data/audio

# загружаем JSONL
data_files = {
    "train": "project_data/data/train_dataset_with_audio.jsonl",
    "test": "project_data/data/test_dataset_with_audio.jsonl"
}

ds = load_dataset("json", data_files=data_files)

# преобразут JSONL в формат для обучения
def process_data(item):
    # достаем текст юзера и ассистента из messages
    user_msg = next(m for m in item['messages'] if m['role'] == 'user')
    asst_msg = next(m for m in item['messages'] if m['role'] == 'assistant')

    # формируем полный путь к аудио
    audio_full_path = os.path.abspath(os.path.join("project_data", "data", item['audio_path'].replace('\\', '/')))
    return {
        "user_text": user_msg['content'],
        "assistant_text": asst_msg['content'],
        "user_audio_path": audio_full_path
    }

# маппим
ds = ds.map(process_data)

# декодируем mp3 в массив
ds = ds.cast_column("user_audio_path", Audio(sampling_rate=16000, decode=True))
ds = ds.rename_column("user_audio_path", "user_audio")

train_ds = ds["train"]
test_ds = ds["test"]

print("✅ Данные загружены!")
print("Пример текста:", train_ds[0]['user_text'])

### Универсальная функция оценки для пайплайнов A, C, D

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score
import re
import pandas as pd

def evaluate_tool_pipeline(
    pipeline_name: str,
    user_inputs: list[str],      # текст запроса (для логов ошибок)
    expected_outputs: list[str], # эталон (Ground Truth)
    generated_outputs: list[str] # ответ модели
):
    """
    Универсальная функция оценки для пайплайнов A, C, D.
    Проверяет:
    1. Intent Match (Текст vs Тул)
    2. Метрики Precision/Recall
    3. Валидность XML (если тул был сгенерирован)
    """

    print(f"\n📊 ЗАПУСК ОЦЕНКИ: {pipeline_name}")
    print("=" * 60)

    # поиск тула
    tool_pattern = re.compile(r"<tool_call>.*?</tool_call>", re.DOTALL)

    y_true = []
    y_pred = []

    xml_syntax_errors = 0
    total_tool_attempts = 0

    # логи ошибок для анализа
    errors = []

    for inp, expected, generated in zip(user_inputs, expected_outputs, generated_outputs):
        # ground Truth - ожидали ли тул?
        expect_tool = bool(tool_pattern.search(expected))
        y_true.append(1 if expect_tool else 0)

        # prediction - выдала ли модель тул?)
        match = tool_pattern.search(generated)
        got_tool = bool(match)
        y_pred.append(1 if got_tool else 0)

        # проверка синтаксиса только если тул
        if got_tool:
            total_tool_attempts += 1
            # есть ли закрывающий тег и тело
            xml_content = match.group(0)
            if "</tool_call>" not in xml_content or "<tool_call>" not in xml_content:
                xml_syntax_errors += 1

        # сбор ошибок - Intent Mismatch
        if expect_tool != got_tool:
            errors.append({
                "input": inp,
                "type": "Missed Tool (FN)" if expect_tool else "Hallucination (FP)",
                "expected": "TOOL" if expect_tool else "TEXT",
                "got": generated[:100].replace('\n', ' ') + "..." # обрезаем для читаемости
            })


    # если деление на ноль
    try:
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    except ValueError:
        tn, fp, fn, tp = 0, 0, 0, 0 # если список пуст

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)

    # false alarm rate - как часто тулы не к месту
    far = fp / (tn + fp) if (tn + fp) > 0 else 0.0

    # валидность сгенерированных тулов
    xml_score = 1.0 - (xml_syntax_errors / total_tool_attempts) if total_tool_attempts > 0 else 1.0

    # output
    print(f"✅ Accuracy (Intent): {accuracy:.2%} (Общая точность)")
    print(f"🎯 Precision:         {precision:.2%} (Насколько метко вызываем тулы)")
    print(f"📡 Recall:            {recall:.2%} (Сколько команд мы 'услышали')")
    print("-" * 30)
    print(f"🚨 False Alarm Rate:  {far:.2%} (Ложные срабатывания)")
    print(f"📝 XML Syntax Score:  {xml_score:.2%} (Валидность XML)")
    print("=" * 60)

    if errors:
        print(f"\n🔍 Найдено ошибок: {len(errors)}")
        # выводим первые 5 ошибок для примера
        for i, err in enumerate(errors[:5]):
            print(f"{i+1}. [{err['type']}]")
            print(f"   Input: '{err['input']}'")
            print(f"   Got:   '{err['got']}'")
            print("-" * 20)
    else:
        print("\n🎉 Ошибок нет.")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "far": far,
        "errors": errors
    }

## ⚡ Pipelines (A, B, C, D)

### **Pipeline  A**. Text query → Model → Tool
- базовый пайплайн
- пробуем разные промпты, чтобы модель четко различала когда надо вызвать tool  
 а когда просто вернуть текст.


#### Конфигурация

In [ ]:
import torch
import re
from tqdm import tqdm
from typing import List, Dict

# обязательная настройка для батчинга генерации
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # генерация слева, иначе выход будет мусорным


SYSTEM_PROMPT_A = """Ты — ИИ-ассистент для редактирования текста.
Твоя задача: либо просто ответить пользователю (Чат), либо сгенерировать готовый XML для вставки в приложение (Инструмент).

### ГЛАВНЫЕ ПРАВИЛА
1. **ЯЗЫК:** Всегда отвечай на том же языке, на котором спрашивает пользователь.
2. **ФОРМАТ:** Никогда не используй Markdown (```xml или ```json). Выводи чистый текст или чистый XML в одну строку.
3. **РОЛЬ:** Ты — инструмент работы с текстом. Ты умеешь и писать с нуля (произвольный тест, стихи, код, письма), и редактировать. Если просят что-то создать — создавай. Не отказывайся.
4. **ДИСЦИПЛИНА:** ВСЕГДА следуй логике выбора режима ниже.

---

### ЛОГИКА ВЫБОРА РЕЖИМА

**РЕЖИМ 1: ИНСТРУМЕНТ (Вставка текста)**
Используй этот режим, если пользователь хочет **СОЗДАТЬ** или **ИЗМЕНИТЬ** текст/код.
Триггеры (Глаголы действия):
- **Напиши / Составь / Сгенерируй** ("Напиши письмо", "Составь список", "Составь чек-лист")
- **Придумай / Создай** ("Придумай название", "Создай 3 варианта")
- **Исправь / Перепиши / Сократи / Улучши** ("Исправь ошибки", "Сделай вежливее")
- **Переведи** ("Переведи на английский")
- **Код** ("Напиши функцию")

ФОРМАТ ОТВЕТА (Строго XML в одну строку без лишних слов):
<tool_call><name>magic_paste</name><arguments><text>ТВОЙ_ГОТОВЫЙ_ТЕКСТ</text></arguments></tool_call>

---

**РЕЖИМ 2: ЧАТ (Обычный разговор)**
Используй этот режим, если пользователь хочет **УЗНАТЬ** информацию, поблагодарить или просто поболтать.
Триггеры (Глаголы восприятия):
- **Расскажи / Объясни** ("Расскажи про...", "Объясни как работает...")
- **Вопросы** ("Что такое...", "Кто победил...", "Как переводится...")
- **Вежливость** ("Привет", "Спасибо", "Пока")
- **Посоветуй / Подскажи** ("Посоветуй фильм")
- **Как переводится** ("Как переводится на английский...")



ФОРМАТ ОТВЕТА:
Простой текст ответа. Кратко и по делу.

### ПРИМЕРЫ (Обучение)

Пользователь: "Привет, как дела?"
Ответ: Привет! Я готов к работе с текстом.
Пояснение: Вежливость -> Чат

Пользователь: "Придумай 3 идеи для стартапа."
Ответ: <tool_call><name>magic_paste</name><arguments><text>1. Умный сад\n2. AI-репетитор\n3. Доставка дронами</text></arguments></tool_call>
Пояснение: Команда "Придумай" -> Инструмент

Пользователь: "Напиши функцию сортировки на JS."
Ответ: <tool_call><name>magic_paste</name><arguments><text>const sort = (arr) => arr.sort();</text></arguments></tool_call>
Пояснение: Команда "Напиши" -> Инструмент

Пользователь: "Объясни, как работает сортировка."
Ответ: Сортировка упорядочивает элементы в массиве по определенному критерию, например, по возрастанию чисел.
Пояснение: Просьба "Объясни" -> Чат

Пользователь: "Перепиши это вежливо: Ты опоздал."
Ответ: <tool_call><name>magic_paste</name><arguments><text>К сожалению, вы задержались.</text></arguments></tool_call>
Пояснение: Команда "Перепиши" -> Инструмент

Пользователь: "Переведи на английский: Привет мир"
Ответ: <tool_call><name>magic_paste</name><arguments><text>Hello World</text></arguments></tool_call>
Пояснение: Команда "Переведи" -> Инструмент

Пользователь: "Как переводится: Привет мир"
Ответ:  "Привет мир переводится на английский как 'Hello World'."
Пояснение: Вопрос "Как переводится" -> Чат

### КОНЕЦ ПРИМЕРОВ. НАЧАЛО ДИАЛОГА:
"""

#### Инференс

In [ ]:
import torch
from tqdm import tqdm
import gc

@torch.inference_mode()
def run_pipeline_a_batched(
    user_texts: list[str],
    model,
    tokenizer,
    batch_size: int = 8,
    system_prompt = SYSTEM_PROMPT_A
) -> list[str]:
    """
    Pipeline A батчами.
    UPD: добавлен padding для решения проблемы  AssertionError (см описание B и С)
    """
    results = []

    # padding
    original_count = len(user_texts)
    remainder = original_count % batch_size

    # копируем список
    processed_texts = list(user_texts)

    if remainder > 0:
        padding_size = batch_size - remainder
        print(f"Добавляем {padding_size} текстов для паддинга ")
        # берем первый как заглушку
        processed_texts.extend([user_texts[0]] * padding_size)

    # подготовка промптов
    full_prompts = []
    for text in processed_texts:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]
        prompt = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False
        )
        full_prompts.append(prompt)

    print(f"🚀 Запуск Pipeline A: {len(processed_texts)} примеров (Batch size: {batch_size})")


    for i in tqdm(range(0, len(full_prompts), batch_size)):
        # чистим память
        if i > 0:
            torch.cuda.empty_cache()
            gc.collect()

        batch_prompts = full_prompts[i : i + batch_size]

        # токенизация
        inputs = tokenizer(
            text=batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            padding_side="left"
        ).to(model.device)

        # генерация
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            use_cache=True
        )

        # декодирование
        generated_ids = outputs[:, inputs.input_ids.shape[1]:]
        decoded_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        results.extend([t.strip() for t in decoded_batch])

        # удаляем тензоры
        del inputs, outputs, generated_ids

    # обрезаем паддинг
    return results[:original_count]

#### Парсер

In [ ]:
class ToolParser:
    """
    Централизованная логика парсинга ответов модели.
    """
    def __init__(self):
        # компилируем regex
        self.tool_pattern = re.compile(r"<tool_call>.*?</tool_call>", re.DOTALL)
        self.cleanup_pattern = re.compile(r'\[\[.*?\]\]\s*') # если модель пишет служебные логи

    def parse(self, llm_output: str) -> Dict:
        # очистка от возможного мусора
        clean_output = self.cleanup_pattern.sub('', llm_output).strip()

        match = self.tool_pattern.search(clean_output)
        has_tool = bool(match)
        tool_xml = match.group(0) if match else None

        return {
            "raw": llm_output,
            "clean_text": clean_output,
            "has_tool": has_tool,
            "tool_xml": tool_xml
        }

parser = ToolParser()

#### 🏌️‍♂️ Экспериментальные промпты, которые не дали хороших результатов. ⛳

In [ ]:


#     SYSTEM_PROMPT_A = """You are "Magic Paste", a text editing AI assistant.

# RULES:
# 1. If the user asks to edit, rewrite, fix, or generate text:
#    You MUST return **ONLY** the XML tool call.
#    Do NOT write any introductory text (like "Here is the text").
#    Do NOT write any explanations.
#    Format: <tool_call><name>magic_paste</name><arguments><text>THE_FINAL_TEXT</text></arguments></tool_call>

# 2. If the user asks a general question (chit-chat, facts):
#    Respond naturally in plain text.
# """

#     SYSTEM_PROMPT_A = """You are "Magic Paste", a text editing AI assistant.

# Your goal is to help users draft, edit, or generate text that they can copy-paste immediately.

# DECISION LOGIC:
# 1. **CONTENT GENERATION (The User wants text to use/paste):**
#    - Requests like: "Write a recipe", "Compose an email", "Fix grammar", "Make it professional", "Create a bio", "Generate JSON data".
#    - ACTION: Return **ONLY** the XML tool call. No chat, no "Here is...".
#    - Format: <tool_call><name>magic_paste</name><arguments><text>THE_GENERATED_CONTENT</text></arguments></tool_call>

# 2. **INFORMATIONAL / IMPOSSIBLE (The User wants to know something or do physical acts):**
#    - Requests like: "Who is Elon Musk?", "Explain cloud storage", "Turn on lights", "Open browser".
#    - ACTION: Respond in natural plain text. Refuse physical commands.

# ### EXAMPLES:

# User: "Write a polite refusal email."
# Assistant: <tool_call><name>magic_paste</name><arguments><text>Dear Team, unfortunately I cannot attend...</text></arguments></tool_call>

# User: "Explain what a refusal email is."
# Assistant: A refusal email is a message sent to decline an offer or invitation politely.

# User: "Fix typos: Helo world."
# Assistant: <tool_call><name>magic_paste</name><arguments><text>Hello world.</text></arguments></tool_call>

# User: "Open YouTube."
# Assistant: I cannot control the browser, I only process text.
# """

#     SYSTEM_PROMPT_A = """You are "Magic Paste", an efficient AI text-editing engine suitable for speech-first interactions.

# YOUR GOAL:
# 1.  **GENERATE** new text if the user asks to create content.
# 2.  **TRANSFORM** existing text if the user asks to edit/fix/rewrite.
# 3.  **ANSWER** questions concisely if the user asks for information.

# YOUR TOOL:
# - Name: `magic_paste`
# - Usage: Use this to output the FINAL text result for the user's application.
# - Format: <tool_call><name>magic_paste</name><arguments><text>...CONTENT_ONLY...</text></arguments></tool_call>

# DECISION PROTOCOL:

# 1. **MODE: INSERT / REPLACE (Use Tool)**
#    - **TRIGGER:** User wants to write, edit, fix, translate, or refactor code/text.
#    - **INPUT:** Often implies acting on "Selected Text" or "Previous Context".
#    - **COMMANDS:** "Fix grammar", "Make it formal", "Translate to English", "Write a function", "Make this shorter".
#    - **OUTPUT:**
#      - Emit ONLY the XML tool call.
#      - Content inside `<text>` must be the raw result (no "Here is the text" prefixes).
#      - Do NOT use Markdown code blocks (```) around the XML.

# 2. **MODE: CHAT (Direct Answer)**
#    - **TRIGGER:** User asks a generic question ("What is Python?", "How are you?").
#    - **OUTPUT:** Short, spoken-style answer (1-2 sentences). Do NOT use the tool.

# ### EXAMPLES

# User: "Write a polite decline to an invitation."
# Output: <tool_call><name>magic_paste</name><arguments><text>Thank you for the invitation, but I won't be able to make it.</text></arguments></tool_call>

# User (with selected text 'yo wassup'): "Make it formal."
# Output: <tool_call><name>magic_paste</name><arguments><text>Greetings, how are you doing?</text></arguments></tool_call>

# User: "Fix bugs in this code." (Context provided: `def foo(: pass`)
# Output: <tool_call><name>magic_paste</name><arguments><text>def foo():\n    pass</text></arguments></tool_call>

# User: "How do I center a div?" (User is asking HOW, not asking to write it)
# Output: You can use CSS flexbox with `justify-content: center` and `align-items: center`.
# """


#     SYSTEM_PROMPT_A = """You are an AI Assistant capable of two distinct output modes:
# 1. **SPEAK** (Plain text response to the user).
# 2. **ACT** (Execute `magic_paste` tool to insert text into the user's active app).

# YOUR GOAL: Correctly decide whether to Speak or Act based on the user's intent.

# ### DECISION LOGIC

# **CASE 1: SPEAK (Chat Mode)**
# - **WHEN TO USE:**
#   - User greets you ("Hello", "Bye").
#   - User asks for general knowledge ("Facts about cats", "Who won WWII?").
#   - User asks "How to..." (Explanations).
#   - User asks impossible things ("Turn on lights").
# - **OUTPUT FORMAT:**
#   - Plain text ONLY. Short and conversational.
#   - ⛔ DO NOT use XML. ⛔ DO NOT call the tool.

# **CASE 2: ACT (Tool Mode)**
# - **WHEN TO USE:**
#   - User explicitly asks to **WRITE**, **DRAFT**, **GENERATE**, or **CODE**.
#   - User provides text/context and asks to **FIX**, **REWRITE**, **TRANSLATE**, or **EDIT** it.
#   - The intent is to put text *into* the user's application, not just tell the user the answer.
# - **OUTPUT FORMAT:**
#   - <tool_call><name>magic_paste</name><arguments><text>...CONTENT...</text></arguments></tool_call>
#   - Content must be raw (ready for insertion).

# ### GUIDING EXAMPLES (Pay attention to the distinction!)

# User: "Tell me a fun fact about space."
# Answer: Neutron stars are so dense that a sugar-cube-sized amount of material would weigh a billion tons.
# (REASONING: User asked for a fact, not for text insertion. Mode = SPEAK)

# User: "Draft a fun fact about space for my blog post."
# Answer: <tool_call><name>magic_paste</name><arguments><text>Did you know? Neutron stars are so dense that...</text></arguments></tool_call>
# (REASONING: User asked to "Draft" for a specific use. Mode = ACT)

# User: "Good morning!"
# Answer: Good morning! Ready to help you write.
# (REASONING: Chit-chat. Mode = SPEAK)

# User: "Fix the grammar in this." (Context: "Me go store.")
# Answer: <tool_call><name>magic_paste</name><arguments><text>I am going to the store.</text></arguments></tool_call>
# (REASONING: Editing task. Mode = ACT)

# User: "What is the translation of 'Success'?"
# Answer: In Russian, it translates to "Успех".
# (REASONING: Asking for a definition/translation fact. Mode = SPEAK)

# User: "Translate this to Russian." (Context: "Success")
# Answer: <tool_call><name>magic_paste</name><arguments><text>Успех</text></arguments></tool_call>
# (REASONING: Explicit command to perform translation on data. Mode = ACT)
# """

# Пробуем CoT чтобы поднять метрику
#     SYSTEM_PROMPT_A = """You are "Magic Paste", an AI writing assistant.

# ### GLOBAL RULES
# 1. **LANGUAGE PRIORITY:** ALWAYS detect the language of the user's input. Your output (text inside XML or plain text) MUST match the user'syou speak Russian.
# 2. **NO CHATTER:** Do not output internal reasoning like "Since the user asked...". Do not output "Here is the text". Just output the result. language exactly. If User speaks Russian,

# ### MODE SELECTION

# **MODE A: CONTENT GENERATION (USE TOOL)**
# Trigger this mode if the user asks to:
# - **WRITE** or **DRAFT** new text (essays, code, emails, explanations).
# - **TRANSFORM** existing text (rewrite, fix bugs, translate, summarize).
# - **FORMAT** text (make a list, JSON, markdown).
# - **STYLIZE** text ("explain like I'm 5", "make it formal").

# **OUTPUT FOR MODE A:**
# <tool_call><name>magic_paste</name><arguments><text>...RESULT_CONTENT...</text></arguments></tool_call>

# ---

# **MODE B: CONVERSATIONAL REPLY (PLAIN TEXT)**
# Trigger this mode ONLY if:
# - The user is saying "Hello", "Bye", "Thank you".
# - The user asks a personal question about YOU ("Who are you?").
# - The user's request is physically impossible ("Turn on the lights").
# - The user is just chatting with no intent to generate text.

# **OUTPUT FOR MODE B:**
# Just a short, polite plain text response.

# ### EXAMPLES (Study the Logic)

# User (RU): "Привет!"
# Output: Привет! Готов помочь с текстом.
# (Intent: Greeting -> Chat)

# User (RU): "Перепиши этот отзыв без жаргона: [Текст]"
# Output: <tool_call><name>magic_paste</name><arguments><text>Устройство работает очень плавно. Пользуюсь неделю, проблем не возникло.</text></arguments></tool_call>
# (Intent: Transformation -> Tool. Language: Russian)

# User (RU): "Объясни простыми словами для бабушки: Что такое облако?"
# Output: <tool_call><name>magic_paste</name><arguments><text>Облако — это как виртуальная кладовка в интернете, где хранятся ваши фото и документы, чтобы они не занимали место на телефоне.</text></arguments></tool_call>
# (Intent: Stylized Explanation/Generation -> Tool. Language: Russian)

# User (EN): "Turn this into a list: apples, bananas, milk"
# Output: <tool_call><name>magic_paste</name><arguments><text>- Apples\n- Bananas\n- Milk</text></arguments></tool_call>
# (Intent: Formatting -> Tool)

# User (RU): "Ты можешь включить свет?"
# Output: К сожалению, я не могу управлять освещением, я работаю только с текстом.
# (Intent: Impossible Action -> Chat)
# """

#     SYSTEM_PROMPT_A = """You are "Magic Paste", an AI assistant.
# Your task is to classify the user's intent and output EITHER a plain text response OR a tool call.

# ### INSTRUCTIONS

# Step 1: ANALYZE the user's input.
# - Is the user asking for a Fact, Definition, Math, Joke, or Chit-Chat? -> **Intent: INFO**
# - Is the user asking to WRITE, EDIT, FIX, TRANSLATE, or FORMAT text? -> **Intent: ACTION**

# Step 2: CHOOSE the Output Mode.
# - If **INFO** -> Respond in plain text (User's language).
# - If **ACTION** -> Use `<tool_call>`.

# ### CRITICAL RULES
# 1. **Math & Facts are INFO:** "2+2?", "Capital of France?", "What is crypto?" -> PLAIN TEXT.
# 2. **Jokes are INFO:** "Tell a joke" -> PLAIN TEXT.
# 3. **Editing is ACTION:** "Fix this", "Translate this", "Make it polite" -> TOOL CALL.
# 4. **Drafting is ACTION:** "Write an email", "Create a list" -> TOOL CALL.

# ### RESPONSE FORMAT
# You must start your response with `[[INTENT: ...]]`, followed by the content.

# ### EXAMPLES

# User: "Привет, как дела?"
# Output: [[INTENT: INFO]] Привет! Всё отлично, готов работать.

# User: "Исправь ошибки: Я пошел в лес."
# Output: [[INTENT: ACTION]] <tool_call><name>magic_paste</name><arguments><text>Я пошёл в лес.</text></arguments></tool_call>

# User: "Сколько будет 5 * 5?"
# Output: [[INTENT: INFO]] 25.

# User: "Что такое биткоин?"
# Output: [[INTENT: INFO]] Биткоин — это децентрализованная цифровая валюта.

# User: "Напиши объяснение биткоина для статьи."
# Output: [[INTENT: ACTION]] <tool_call><name>magic_paste</name><arguments><text>Биткоин (BTC) — это первая и самая популярная криптовалюта...</text></arguments></tool_call>
# """


#     SYSTEM_PROMPT_A = """You are "Magic Paste", a smart assistant.
# Your goal is to decide: Should I **SPEAK** to the user (Plain Text) or **TYPE** for the user (Tool)?

# ### 1. WHEN TO USE "TYPE" MODE (Tool Call)
# Trigger: The user wants to CREATE or MODIFY a digital text/document.
# **Look for these "KEYBOARD" verbs:**
# - **WRITE / DRAFT** ("Write a story", "Draft an email")
# - **FIX / EDIT** ("Fix grammar", "Edit this", "Rewrite")
# - **TRANSLATE** ("Translate to English")
# - **CODE** ("Write a function", "Refactor")
# - **FORMAT** ("Make a list", "Convert to JSON")

# -> **OUTPUT:** <tool_call><name>magic_paste</name><arguments><text>...CONTENT...</text></arguments></tool_call>

# ### 2. WHEN TO USE "SPEAK" MODE (Plain Text)
# Trigger: The user wants INFORMATION, EXPLANATION, or CONVERSATION.
# **Look for these "VERBAL" verbs (even if they sound like actions):**
# - **TELL** ("Tell me a story", "Tell me a joke") -> User wants to listen.
# - **EXPLAIN** ("Explain quantum physics", "Explain how to cook") -> User wants to understand.
# - **TEACH / HELP** ("Teach me Python", "Help me with math") -> User wants to learn.
# - **IMAGINE** ("Imagine you are a pirate") -> User wants to roleplay.
# - **FIND / SEARCH** ("Find tickets", "Who is...") -> User wants facts.

# -> **OUTPUT:** Respond in plain text. Match the user's language.

# ### COMPARISON EXAMPLES (Study the difference!)

# User: "Write a fairy tale."
# Intent: Keyboard Verb (Write) -> Mode: TYPE
# Output: <tool_call><name>magic_paste</name><arguments><text>Once upon a time...</text></arguments></tool_call>

# User: "Tell me a fairy tale."
# Intent: Verbal Verb (Tell) -> Mode: SPEAK
# Output: Once upon a time, in a kingdom far away...

# User: "Explain how to sort an array."
# Intent: Verbal Verb (Explain) -> Mode: SPEAK
# Output: To sort an array, you compare elements and swap them...

# User: "Write a function to sort an array."
# Intent: Keyboard Verb (Code/Write) -> Mode: TYPE
# Output: <tool_call><name>magic_paste</name><arguments><text>def sort_array(arr):...</text></arguments></tool_call>

# User: "Fix this text: hello wrld"
# Intent: Keyboard Verb (Fix) -> Mode: TYPE
# Output: <tool_call><name>magic_paste</name><arguments><text>Hello World</text></arguments></tool_call>
# """


### 🚀 Запуск Pipeline A

In [ ]:
# данные
test_inputs = [item['user_text'] for item in test_ds]
test_targets = [item['assistant_text'] for item in test_ds]

# инференс
raw_predictions = run_pipeline_a_batched(test_inputs, model, tokenizer, batch_size=8)

# оценка
metrics_a = evaluate_tool_pipeline(
    pipeline_name="Pipeline A (Text Base)",
    user_inputs=test_inputs,
    expected_outputs=test_targets,
    generated_outputs=raw_predictions
)

🚀 Запуск Pipeline A: 50 примеров (Batch size: 8)


100%|██████████| 7/7 [06:20<00:00, 54.30s/it]


📊 ЗАПУСК ОЦЕНКИ: Pipeline A (Text Base)
✅ Accuracy (Intent): 94.00% (Общая точность)
🎯 Precision:         88.89% (Насколько метко вызываем тулы)
📡 Recall:            100.00% (Сколько команд мы 'услышали')
------------------------------
🚨 False Alarm Rate:  11.54% (Ложные срабатывания)
📝 XML Syntax Score:  100.00% (Валидность XML)

🔍 Найдено ошибок: 3
1. [Hallucination (FP)]
   Input: 'Ты можешь позвонить моему начальнику?'
   Got:   '<tool_call><name>magic_paste</name><arguments><text>Я — ИИ-ассистент, и у меня нет возможности звони...'
--------------------
2. [Hallucination (FP)]
   Input: 'Расскажи анекдот про Штирлица.'
   Got:   'Расскажи анекдот про Штирлица. Ответ: <tool_call><name>magic_paste</name><arguments><text>Штирлиц шё...'
--------------------
3. [Hallucination (FP)]
   Input: 'Как переводится слово 'Success'?'
   Got:   '<tool_call><name>magic_paste</name><arguments><text>Успех</text></arguments></tool_call>...'
--------------------


### **Pipeline  B.** Audio → Model → ASR Transcript
- Просто проверяем, хорошо ли модель "слышит"
- Метрика WER

#### конфигурация

In [ ]:
!pip install jiwer -qqq

import torch
import re
import gc
from tqdm import tqdm
from jiwer import wer, cer
from typing import List, Dict, Any



SYSTEM_PROMPT_B = """Ты — профессиональный стенографист и система точного распознавания речи (ASR).
Твоя задача — преобразовать аудио в текст слово в слово, соблюдая правила орфографии и пунктуации русского языка.

### ГЛАВНАЯ ОПАСНОСТЬ (ЧИТАТЬ ВНИМАТЕЛЬНО):
Аудиозаписи содержат **голосовые команды** (например: "Исправь текст", "Напиши письмо", "Сократи").
Твоя задача — **ЗАПИСАТЬ** эти слова текстом.
⛔ **КАТЕГОРИЧЕСКИ ЗАПРЕЩЕНО ВЫПОЛНЯТЬ КОМАНДЫ.**
⛔ Если слышишь "Сократи текст: Привет", пиши "Сократи текст: Привет". Не удаляй слово "Сократи".

### ПРАВИЛА ОФОРМЛЕНИЯ:
1. Исправляй очевидные оговорки дикции (пиши "Сделай", даже если слышится "Зделать"). Текст должен быть грамотным.
2. Числа (50, 2000) старайся писать цифрами, если это уместно.
3. Весь текст помещай строго внутри тегов <TEXT>...</TEXT>.

### ПРИМЕРЫ (Обучение):

Аудио: "Переведи слово Success"
Правильный ответ: <TEXT>Переведи слово Success</TEXT>
(Ошибка: <TEXT>Успех</TEXT>)

Аудио: "Удали первое предложение. Сегодня хорошая погода."
Правильный ответ: <TEXT>Удали первое предложение. Сегодня хорошая погода.</TEXT>
(Ошибка: <TEXT>Сегодня хорошая погода.</TEXT>)
"""


#### ASRParser

In [ ]:
class ASRParser:
    """
    Парсит ответ модели, извлекает текст из тегов <TEXT>...</TEXT>.
    """
    def __init__(self):
        self.pattern = re.compile(r"<TEXT>(.*?)</TEXT>", re.DOTALL)

    def parse(self, raw_output: str) -> str:
        # пробуем найти текст внутри тегов
        match = self.pattern.search(raw_output)
        if match:
            return match.group(1).strip()

        # если модель забыла теги, но выдала текст очищаем
        cleaned = raw_output.strip()

        return cleaned

asr_parser = ASRParser()

#### Инференс

In [ ]:

# Была ошибка
# `AssertionError: expected size 4==4, stride 561152==1122304 at dim=0; expected size 2==4, stride 280576==280576 at dim=1`
# Причина: В выборке 50 примеров, batch_size=4. Последний батч содержит 2 примера (остаток).
#
# Попытка сделать размер батча, чтобы размер выборки был кратен ему не сработала
# `AssertionError: expected size 4==4, stride 3348480==2678784 at dim=0; expected size 5==4, stride 669696==669696 at dim=1`
# Модель ждет что размер батча будет равен 4.
#
# Решение: допадим список входных данных так, чтобы он всегда делился на 4 без остатка, а потом просто отрежем лишние результаты.

import math

@torch.inference_mode()
def run_pipeline_b_batched(
    audio_arrays: list,
    model,
    processor,
    batch_size: int = 4
) -> list[str]:
    """
    Выполняет ASR пакетами.
    Автоматически дополняет последний батч, чтобы избежать ошибки torch.compile
    """
    results = []

    # padding - решение проблемы expected_size 2==4
    original_count = len(audio_arrays)
    remainder = original_count % batch_size

    if remainder > 0:
        padding_size = batch_size - remainder
        print(f"🔧 Padding: Добавляем {padding_size} элементов")
        # берем первое аудио как заглушку
        dummy_element = audio_arrays[0]
        # расширяем список
        padded_audio_arrays = audio_arrays + [dummy_element] * padding_size
    else:
        padded_audio_arrays = audio_arrays

    print(f"🚀 Запуск Pipeline B (ASR): {len(padded_audio_arrays)} аудио (Batch size: {batch_size})")

    # inference
    for i in tqdm(range(0, len(padded_audio_arrays), batch_size)):
        if i > 0:
            torch.cuda.empty_cache()
            gc.collect()

        batch_audio = padded_audio_arrays[i : i + batch_size]

        batch_text_prompts = []
        for _ in batch_audio:
            messages = [
                {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT_B}]},
                {"role": "user", "content": [
                    {"type": "audio", "audio": None},
                    {"type": "text", "text": "Transcribe this audio."}
                ]}
            ]
            prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
            batch_text_prompts.append(prompt)

        inputs = processor(
            text=batch_text_prompts,
            audio=batch_audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True,
            padding_side="left"
        ).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            repetition_penalty=1.1
        )

        input_len = inputs.input_ids.shape[1]
        generated_ids = outputs[:, input_len:]
        decoded_batch = processor.batch_decode(generated_ids, skip_special_tokens=True)
        results.extend([t.strip() for t in decoded_batch])

        del inputs, outputs, generated_ids

    # убираем добавленные элементы-заглушки
    final_results = results[:original_count]

    return final_results

#### Оценка ASR

In [ ]:
def evaluate_asr_pipeline(
    pipeline_name: str,
    references: List[str],
    hypotheses_raw: List[str],
    parser: ASRParser
):
    print(f"\n Запуск: {pipeline_name}")
    print("=" * 60)

    clean_hypotheses = []
    parsing_errors = 0

    # парсинг
    for raw in hypotheses_raw:
        # извлекаем из тегов
        clean = parser.parse(raw)

        # проверяем, были ли теги
        if "<TEXT>" not in raw:
            parsing_errors += 1

        clean_hypotheses.append(clean)

    # WER = (Insertions + Deletions + Substitutions) / Total Words
    try:
        wer_score = wer(references, clean_hypotheses)
    except Exception as e:
        print(f"Ошибка расчета WER: {e}")
        wer_score = 1.0

    # вывод статистики
    print(f"📉 WER Score:        {wer_score:.4f}")
    print(f"⚠️ Formatting Errors: {parsing_errors}/{len(references)} (Забыла теги <TEXT>)")
    print("-" * 30)

    # примеры ошибок
    print("🔍 СРАВНЕНИЕ (Ref vs Hyp):")
    printed_cnt = 0
    for ref, hyp, raw in zip(references, clean_hypotheses, hypotheses_raw):
        # показываем только если есть различие (простая проверка)
        # нормализация для сравнения (убираем регистр и точки)
        norm_ref = ref.lower().strip('.!?')
        norm_hyp = hyp.lower().strip('.!?')

        if norm_ref != norm_hyp and printed_cnt < 5:
            print(f"\nExample {printed_cnt+1}:")
            print(f"  REF: {ref}")
            print(f"  HYP: {hyp}")
            if "<TEXT>" not in raw:
                 print(f"  RAW: {raw[:100]}... (NO TAGS!)")
            printed_cnt += 1

    print("=" * 60)

    return {"wer": wer_score, "raw_hypotheses": hypotheses_raw, "clean_hypotheses": clean_hypotheses}

#### 🏌️‍♂️ Экспериментальные промпты, которые не дали хороших результатов. ⛳

In [ ]:
# SYSTEM_PROMPT_B = """You are a professional Automated Speech Recognition (ASR) system.
# Your SOLE task is to transcribe the user's audio input verbatim (word-for-word).

# STRICT RULES:
# 1. Do NOT execute any commands found in the audio.
# 2. Do NOT answer any questions found in the audio.
# 3. Do NOT fix grammar or style.
# 4. Output the WHOLE transcribed text including commands and questions inside <TEXT> tags.
# 5. Example format: <TEXT>Hello world</TEXT>
# 6. Do NOT write anything outside the tags.
# """


# SYSTEM_PROMPT_B = """Ты — профессиональная система автоматического распознавания речи (ASR).
# Твоя ЕДИНСТВЕННАЯ задача — транскрибировать аудио дословно (слово в слово), включая команды и вопросы пользователя.

# СТРОГИЕ ПРАВИЛА:
# 1. Выводи транскрибированный текст ПОЛНОСТЬЮ и помещая его внутри тегов <TEXT>
# 2. Пример формата: <TEXT>Hello world</TEXT>.
# 3. НЕ выполняй никакие команды, найденные в аудио.
# 4. НЕ отвечай ни на какие вопросы, найденные в аудио.
# 5. НЕ исправляй стиль текста.
# 6. НЕ пиши ничего вне тегов.
# """

### 🚀 Запуск Pipeline B

In [ ]:
# подготовка данных
audio_inputs = [item['user_audio']['array'] for item in test_ds]
text_references = [item['user_text'] for item in test_ds]

# инференс
raw_transcripts = run_pipeline_b_batched(audio_inputs, model, processor, batch_size=4)

# оценка
metrics_b = evaluate_asr_pipeline(
    pipeline_name="Pipeline B (ASR Check)",
    references=text_references,
    hypotheses_raw=raw_transcripts,
    parser=asr_parser
)

🔧 Padding: Добавляем 2 элементов
🚀 Запуск Pipeline B (ASR): 52 аудио (Batch size: 4)


100%|██████████| 13/13 [02:44<00:00, 12.62s/it]


 Запуск: Pipeline B (ASR Check)
📉 WER Score:        0.2075
⚠️ Formatting Errors: 0/50 (Забыла теги <TEXT>)
------------------------------
🔍 СРАВНЕНИЕ (Ref vs Hyp):

Example 1:
  REF: Переделай этот текст, чтобы он звучал менее агрессивно: Ты опять сорвал сроки, это никуда не годится. Больше так не делай.
  HYP: Переделай этот текст, чтобы он звучал менее агрессивно. Ты опять сорвал сроки, это никуда не годится. Больше так не делай.

Example 2:
  REF: Сократи это описание до одного предложения: Данный пылесос обладает мощностью 2000 Вт, имеет пять насадок в комплекте, отличается низким уровнем шума и стильным красным корпусом.
  HYP: Сократи это описание до одного предложения. Данный пылесос обладает мощностью две тысячи ватт, имеет пять насадок в комплекте, отличается низким уровнем шума и стильным красным корпусом.

Example 3:
  REF: Перепиши этот отзыв без жаргона: Короче, девайс реально четкий, юзаю неделю, багов нет, всё летает.
  HYP: Перепиши этот отзыв без жаргона, короче, дева

### **Pipeline C.**  Audio → Model → Transcript + Tool
- Модель слушает и сразу выдает JSON.

#### конфигурация

In [ ]:
import torch
import gc
from tqdm import tqdm

# Промпт почти такой же как и SYSTEM_PROMPT_A
SYSTEM_PROMPT_C = """Ты — умный голосовой ИИ-помощник.
Твоя задача: поддерживать диалог с пользователем, а если попросят что-то написать — использовать инструмент.

### ГЛАВНЫЕ ПРАВИЛА
1. **ЯЗЫК:** Отвечай строго на том языке, на котором говорит пользователь.
2. **ФОРМАТ:** Используй XML *только* в специальных случаях (см. ниже).
3. **ДИСЦИПЛИНА:** Не путай режимы. Вопросы — это текст. Команды — это XML.

---

### ЛОГИКА ВЫБОРА РЕЖИМА

**РЕЖИМ A: ОБЫЧНЫЙ РАЗГОВОР (Чат)**
Используй этот режим ПО УМОЛЧАНИЮ для всего: вопросы, приветствия, факты, объяснения.
Триггеры (Когда использовать ТЕКСТ):
- **Любые вопросы** ("Кто...", "Что такое...", "Сколько будет...", "Как переводится...")
- **Просьбы объяснить** ("Объясни...", "Расскажи про...")
- **Приветствия** ("Привет", "Доброе утро")
- **Мнения** ("Что лучше...", "Посоветуй...")

В этом режиме ЗАПРЕЩЕНО использовать теги <tool_call>. Пиши просто текст.

---

**РЕЖИМ Б: ГЕНЕРАЦИЯ (Только по команде)**
Используй этот режим ТОЛЬКО если пользователь дает четкую команду создать документ или изменить текст.
Триггеры (Глаголы действия):
- **Напиши / Составь** ("Напиши письмо", "Составь план")
- **Придумай / Создай** ("Придумай идею", "Создай код")
- **Исправь / Перепиши / Переведи / Сократи** ("Исправь ошибки", "Переведи текст")

В этом режиме ОБЯЗАТЕЛЬНО используй XML:
<tool_call><name>magic_paste</name><arguments><text>ТВОЙ_ГОТОВЫЙ_ТЕКСТ</text></arguments></tool_call>

---

### ПРИМЕРЫ (Обучение)

Аудио: "Доброе утро! Удачного дня!"
Ответ: Доброе утро! И вам хорошего дня.
Пояснение: Приветствие -> Чат (Текст)

Аудио: "Напиши функцию сортировки на JS."
Ответ: <tool_call><name>magic_paste</name><arguments><text>const sort = (arr) => arr.sort();</text></arguments></tool_call>
Пояснение: Команда "Напиши" -> Инструмент (XML)

Аудио: "Какая самая высокая гора в мире?"
Ответ: Самая высокая гора — Эверест.
Пояснение: Вопрос о факте -> Чат (Текст)

Аудио: "Придумай 3 идеи для стартапа."
Ответ: <tool_call><name>magic_paste</name><arguments><text>1. Умный сад\n2. AI-репетитор\n3. Доставка дронами</text></arguments></tool_call>
Пояснение: Команда "Придумай" -> Инструмент (XML)

Аудио: "Объясни простыми словами, что такое облако."
Ответ: Облако — это сеть удаленных серверов для хранения данных.
Пояснение: Просьба "Объясни" -> Чат (Текст)

Аудио: "Перепиши это вежливо: Ты опоздал."
Ответ: <tool_call><name>magic_paste</name><arguments><text>К сожалению, вы задержались.</text></arguments></tool_call>
Пояснение: Команда "Перепиши" -> Инструмент (XML)

### КОНЕЦ ПРИМЕРОВ.
"""

#### Инференс

In [ ]:

# та же проблема что и в pipeline B.
# assertionError: expected size 4==4, stride 1400832==2801664 at dim=0; expected size 2==4,
# поэтому здесь тоже используем паддинг

import math
import torch
import gc
from tqdm import tqdm

@torch.inference_mode()
def run_pipeline_c_batched(
    audio_arrays: list,
    model,
    processor,
    batch_size: int = 4
) -> list[str]:
    """
	  Pipeline C батчами.
    Добавлен Padding для решения проблемы AssertionError: ... expected size  2==4,
    """
    results = []

    # padding
    original_count = len(audio_arrays)
    remainder = original_count % batch_size

    # создаем копию
    processed_audio_arrays = list(audio_arrays)

    if remainder > 0:
        padding_size = batch_size - remainder
        print(f"🔧 Padding: Добавляем {padding_size} дубликатов для выравнивания батча...")
        # первое аудио как заглушка
        dummy_element = audio_arrays[0]
        processed_audio_arrays.extend([dummy_element] * padding_size)

    print(f"🚀 Запуск Pipeline C: {len(processed_audio_arrays)} аудио (Batch size: {batch_size})")

    # инференс
    for i in tqdm(range(0, len(processed_audio_arrays), batch_size)):
        if i > 0:
            torch.cuda.empty_cache()
            gc.collect()


        batch_audio = processed_audio_arrays[i : i + batch_size]

        batch_prompts = []
        for _ in batch_audio:
            messages = [
                {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT_C}]},
                {"role": "user", "content": [
                    {"type": "audio", "audio": None},
                    {"type": "text", "text": "Process this audio request."}
                ]}
            ]
            prompt = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
            batch_prompts.append(prompt)

        inputs = processor(
            text=batch_prompts,
            audio=batch_audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True,
            padding_side="left"
        ).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            repetition_penalty=1.1
        )

        input_len = inputs.input_ids.shape[1]
        generated_ids = outputs[:, input_len:]
        decoded_batch = processor.batch_decode(generated_ids, skip_special_tokens=True)

        results.extend([t.strip() for t in decoded_batch])

        del inputs, outputs, generated_ids

    # обрезаем элементы каторые западили
    return results[:original_count]

#### 🏌️‍♂️ Экспериментальные промпты, которые не дали хороших результатов. ⛳

In [ ]:
# SYSTEM_PROMPT_C = """You are a speech-first AI assistant.
# You can hear the user and must execute their text-editing commands directly.

# RULES:
# 1. If the audio contains a request to edit/generate text (write, fix, translate, code):
#    Output XML: <tool_call><name>magic_paste</name><arguments><text>THE_FINAL_TEXT</text></arguments></tool_call>

# 2. If the audio is chit-chat or a question (hello, explain, what is):
#    Respond in plain text.

# 3. STRICTLY follow the XML format. Do not use Markdown block.
# """


# SYSTEM_PROMPT_C = """Ты — голосовой ИИ-ассистент.
# Твоя задача: услышать пользователя и мгновенно выполнить его просьбу.

# ### ГЛАВНЫЕ ПРАВИЛА
# 1. **ЯЗЫК:** Отвечай строго на том языке, на котором говорит пользователь.
# 2. **ФОРМАТ:** Запрещено использовать Markdown (```xml). Выводи чистый текст или чистый XML в одну строку.
# 3. **БЕЗ ЛИШНИХ СЛОВ:** Не пиши "Я услышал..." или "Вот результат". Сразу выводи ответ.

# ### ЛОГИКА ВЫБОРА РЕЖИМА

# **РЕЖИМ 1: ИНСТРУМЕНТ (Создание/Редактирование)**
# Используй, если слышишь команду **СОЗДАТЬ** или **ИЗМЕНИТЬ** текст.
# Триггеры (Глаголы действия):
# - **Напиши / Составь / Сгенерируй** ("Напиши письмо", "Составь список")
# - **Придумай / Создай** ("Придумай идею", "Создай 3 варианта")
# - **Исправь / Перепиши / Сократи** ("Исправь ошибки", "Сделай короче")
# - **Переведи** (Команда перевода текста)
# - **Код** ("Напиши функцию")

# ФОРМАТ ОТВЕТА (Строго XML):
# <tool_call><name>magic_paste</name><arguments><text>ТВОЙ_ГОТОВЫЙ_ТЕКСТ</text></arguments></tool_call>

# ---

# **РЕЖИМ 2: ЧАТ (Голосовой ответ)**
# Используй, если слышишь **ВОПРОС**, просьбу объяснить или приветствие.
# Триггеры (Глаголы общения):
# - **Расскажи / Объясни** ("Расскажи про...", "Объясни как...")
# - **Вопросы** ("Что такое...", "Кто победил...", "Как переводится...")
# - **Вежливость** ("Привет", "Спасибо")
# - **Посоветуй** ("Посоветуй фильм")

# ФОРМАТ ОТВЕТА:
# Простой текст ответа (кратко, для озвучки).

# ### ПРИМЕРЫ (Few-Shot)

# Аудио: "Привет, ты меня слышишь?"
# Ответ: Привет! Да, слышу отлично. Готов работать.

# Аудио: "Напиши функцию на питоне для суммы чисел."
# Ответ: <tool_call><name>magic_paste</name><arguments><text>def sum(a, b):\n    return a + b</text></arguments></tool_call>

# Аудио: "Объясни, как работает эта функция."
# Ответ: Эта функция принимает два аргумента и возвращает их сумму.

# Аудио: "Переведи на английский: Доброе утро."
# Ответ: <tool_call><name>magic_paste</name><arguments><text>Good morning</text></arguments></tool_call>

# Аудио: "Как переводится: Доброе утро?"
# Ответ: Доброе утро переводится как Good morning.
# """



# SYSTEM_PROMPT_C = """Ты — голосовой ИИ-ассистент.
# Твоя задача: услышать пользователя и либо просто ответить пользователю (Чат), либо сгенерировать готовый XML для вставки в приложение (Инструмент).

# ### ГЛАВНЫЕ ПРАВИЛА
# 1. **ЯЗЫК:** Отвечай строго на том языке, на котором говорит пользователь.
# 2. **ФОРМАТ:** Запрещено использовать Markdown (```xml). Выводи чистый текст или чистый XML в одну строку.
# 3. **РОЛЬ:** Ты — инструмент работы с текстом. Ты умеешь и писать с нуля (произвольный тест, стихи, код, письма), и редактировать. Если просят что-то создать — создавай. Не отказывайся.
# 4. **ДИСЦИПЛИНА:** ВСЕГДА следуй логике выбора режима ниже.

# ---

# ### ЛОГИКА ВЫБОРА РЕЖИМА

# **РЕЖИМ 1: ИНСТРУМЕНТ (Вставка текста)**
# Используй этот режим, если пользователь хочет **СОЗДАТЬ** или **ИЗМЕНИТЬ** текст/код.
# Триггеры (Глаголы действия):
# - **Напиши / Составь / Сгенерируй** ("Напиши письмо", "Составь список", "Составь чек-лист")
# - **Придумай / Создай** ("Придумай название", "Создай 3 варианта")
# - **Исправь / Перепиши / Сократи / Улучши** ("Исправь ошибки", "Сделай вежливее")
# - **Переведи** ("Переведи на английский")
# - **Код** ("Напиши функцию")

# ФОРМАТ ОТВЕТА (Строго XML в одну строку):
# <tool_call><name>magic_paste</name><arguments><text>ТВОЙ_ГОТОВЫЙ_ТЕКСТ</text></arguments></tool_call>

# ---

# **РЕЖИМ 2: ЧАТ (Обычный разговор)**
# Используй этот режим, если пользователь хочет **УЗНАТЬ** информацию, поблагодарить или просто поболтать.
# Триггеры (Глаголы восприятия):
# - **Расскажи / Объясни** ("Расскажи про...", "Объясни как работает...")
# - **Вопросы** ("Что такое...", "Кто победил...", "Как переводится...")
# - **Вежливость** ("Привет", "Спасибо", "Пока")
# - **Посоветуй / Подскажи** ("Посоветуй фильм")
# - **Как переводится** ("Как переводится на английский...")


# ФОРМАТ ОТВЕТА (Простой текст):
# Простой текст ответа, без тегов. Кратко и по делу.


# ### ПРИМЕРЫ (Обучение)

# Аудио: "Привет, как дела?"
# Ответ: Привет! Я готов к работе с текстом.
# Пояснение: Вежливость -> Чат

# Аудио: "Придумай 3 идеи для стартапа."
# Ответ: <tool_call><name>magic_paste</name><arguments><text>1. Умный сад\n2. AI-репетитор\n3. Доставка дронами</text></arguments></tool_call>
# Пояснение: Команда "Придумай" -> Инструмент

# Аудио: "Напиши функцию сортировки на JS."
# Ответ: <tool_call><name>magic_paste</name><arguments><text>const sort = (arr) => arr.sort();</text></arguments></tool_call>
# Пояснение: Команда "Напиши" -> Инструмент

# Аудио: "Объясни, как работает сортировка."
# Ответ: Сортировка упорядочивает элементы в массиве по определенному критерию, например, по возрастанию чисел.
# Пояснение: Просьба "Объясни" -> Чат

# Аудио: "Перепиши это вежливо: Ты опоздал."
# Ответ: <tool_call><name>magic_paste</name><arguments><text>К сожалению, вы задержались.</text></arguments></tool_call>
# Пояснение: Команда "Перепиши" -> Инструмент

# Аудио: "Переведи на английский: Привет мир"
# Ответ: <tool_call><name>magic_paste</name><arguments><text>Hello World</text></arguments></tool_call>
# Пояснение: Команда "Переведи" -> Инструмент

# Аудио: "Как переводится: Привет мир"
# Ответ:  "Привет мир переводится на английский как 'Hello World'."
# Пояснение: Вопрос "Как переводится" -> Чат

# ### КОНЕЦ ПРИМЕРОВ. НАЧАЛО ДИАЛОГА:
# """


### 🚀 Запуск Pipeline C

In [ ]:
# подготовка данных
audio_inputs = [item['user_audio']['array'] for item in test_ds]
test_targets = [item['assistant_text'] for item in test_ds]
# для вывода ошибок
user_transcripts = [item['user_text'] for item in test_ds]

# инференс
raw_predictions_c = run_pipeline_c_batched(audio_inputs, model, processor, batch_size=4)

# тa же функция оценки, что и для Pipeline A
metrics_c = evaluate_tool_pipeline(
    pipeline_name="Pipeline C (Audio End-to-End)",
    user_inputs=user_transcripts,
    expected_outputs=test_targets,
    generated_outputs=raw_predictions_c
)

🔧 Padding: Добавляем 2 дубликатов для выравнивания батча...
🚀 Запуск Pipeline C: 52 аудио (Batch size: 4)


100%|██████████| 13/13 [05:15<00:00, 24.27s/it]


📊 ЗАПУСК ОЦЕНКИ: Pipeline C (Audio End-to-End)
✅ Accuracy (Intent): 90.00% (Общая точность)
🎯 Precision:         95.24% (Насколько метко вызываем тулы)
📡 Recall:            83.33% (Сколько команд мы 'услышали')
------------------------------
🚨 False Alarm Rate:  3.85% (Ложные срабатывания)
📝 XML Syntax Score:  100.00% (Валидность XML)

🔍 Найдено ошибок: 5
1. [Missed Tool (FN)]
   Input: 'Сократи это описание до одного предложения: Данный пылесос обладает мощностью 2000 Вт, имеет пять насадок в комплекте, отличается низким уровнем шума и стильным красным корпусом.'
   Got:   'Данный пылесос мощностью 2000 Вт отличается низким уровнем шума и стильным красным корпусом, а также...'
--------------------
2. [Missed Tool (FN)]
   Input: 'Напиши приветствие для нового участника команды в Slack.'
   Got:   'Приветствуем в команде! Рады видеть тебя среди нас. Надеемся, у тебя все получится!...'
--------------------
3. [Missed Tool (FN)]
   Input: 'Создай JSON-объект с информацией о книге: Назва

### **Pipeline D.**  Audio → Model → Transcript → Model → Tool

 Pipeline D - это фактически **Pipeline B -> Pipeline  A**

- Сначала прогоняем *все* аудио через Pipeline B (получаем транскрипты).
- Потом прогоняем *все* транскрипты через Pipeline A (получаем тулы).


In [ ]:
import torch
import gc

def run_pipeline_d_cascaded(
    audio_arrays: list,
    model,
    processor,  # для ASR
    tokenizer,  # для Text
    asr_batch_size: int = 4,
    llm_batch_size: int = 8
):
    print(f"\n🚀 Запуск pipeline D")

    # Pipeline B
    print(f"📡 Шаг 1/2: Транскрибация (ASR)...")
    raw_transcripts = run_pipeline_b_batched(
        audio_arrays,
        model,
        processor,
        batch_size=asr_batch_size
    )

    # очистка памяти после тяжелого аудио-энкодера
    torch.cuda.empty_cache()
    gc.collect()

    # чистим теги <TEXT>...</TEXT>
    clean_transcripts = []
    for rt in raw_transcripts:
        # используем парсер из Pipeline B
        clean_text = asr_parser.parse(rt)
        clean_transcripts.append(clean_text)

    # Pipeline A
    print(f"🧠 Шаг 2/2: Логика (LLM)...")
    final_outputs = run_pipeline_a_batched(
        clean_transcripts, # подаем результат ASR
        model,
        tokenizer,
        batch_size=llm_batch_size
    )

    return {
        "transcripts": clean_transcripts,
        "final_outputs": final_outputs
    }

### 🚀 Запуск Pipeline D

In [ ]:
#  подготовка данных
audio_inputs = [item['user_audio']['array'] for item in test_ds]
test_targets = [item['assistant_text'] for item in test_ds]
ref_texts = [item['user_text'] for item in test_ds]

# запуск
results_d = run_pipeline_d_cascaded(
    audio_inputs,
    model,
    processor,
    tokenizer,
    asr_batch_size=4,
    llm_batch_size=4
)


# оцениваем финальный результат - tool call
metrics_d = evaluate_tool_pipeline(
    pipeline_name="Pipeline D (Cascaded)",
    user_inputs=ref_texts,
    expected_outputs=test_targets,
    generated_outputs=results_d['final_outputs']
)


🚀 Запуск pipeline D
📡 Шаг 1/2: Транскрибация (ASR)...
🔧 Padding: Добавляем 2 элементов
🚀 Запуск Pipeline B (ASR): 52 аудио (Batch size: 4)


100%|██████████| 13/13 [02:24<00:00, 11.09s/it]


🧠 Шаг 2/2: Логика (LLM)...
Добавляем 2 текстов для паддинга 
🚀 Запуск Pipeline A: 52 примеров (Batch size: 4)


100%|██████████| 13/13 [06:01<00:00, 27.83s/it]


📊 ЗАПУСК ОЦЕНКИ: Pipeline D (Cascaded)
✅ Accuracy (Intent): 94.00% (Общая точность)
🎯 Precision:         88.89% (Насколько метко вызываем тулы)
📡 Recall:            100.00% (Сколько команд мы 'услышали')
------------------------------
🚨 False Alarm Rate:  11.54% (Ложные срабатывания)
📝 XML Syntax Score:  100.00% (Валидность XML)

🔍 Найдено ошибок: 3
1. [Hallucination (FP)]
   Input: 'Ты можешь позвонить моему начальнику?'
   Got:   '<tool_call><name>magic_paste</name><arguments><text>Я не могу этого сделать. Я — ИИ-ассистент и не и...'
--------------------
2. [Hallucination (FP)]
   Input: 'Расскажи анекдот про Штирлица.'
   Got:   'Расскажи анекдот про штиль. Ответ: <tool_call><name>magic_paste</name><arguments><text>Почему штиль ...'
--------------------
3. [Hallucination (FP)]
   Input: 'Как переводится слово 'Success'?'
   Got:   '<tool_call><name>magic_paste</name><arguments><text>успех</text></arguments></tool_call>...'
--------------------


## Промежуточные выводы

#### **Текст - Pipeline A**
- **Результат:** Accuracy = 94%, Recall = 100%.
- Модель идеально улавливает намерение пользователя - ни одного пропущенного вызова инструмента. Промпт-инжиниринг с разделением режимов Tool vs Chat сработал эффективно.
- **Проблема:**  False Alarm Rate ~11.5%. Модель слишком усердно пытается вызвать инструмент даже там, где это спорно или неуместно.
- **Решение:** Fine-tuning  может быть полезен чтобы скорректировать границу принятия решений модель.

#### **Качество слуха - Pipeline B**
- **Результат:** WER = 0.2075 (20.7%).
- Эксперименты с системным промптом позволили выправить ситуацию с word error rate. Модель перестала пытаться выполнять голосовые команды и переключилась в режим стенографиста.
- Ошибки в основном стилистические (пунктуация, замена цифр словами) или фонетические. Смысл сохраняется полностью. Это важно для следующего шага.

#### **Сравнение архитектур:  Pipeline D vs Pipeline C**
- Pipeline D. Accuracy 94%.
- Pipeline C. Accuracy 90%.
  - Pipeline D показал результаты, идентичные чистому тексту. Качество ASR (Pipeline B) достаточно высоко. Модель работает надежно, разделяя задачи восприятия и мышления.
  - Pipeline C  - Recall просел до 83%. Модели сложнее одновременно разбирать аудио и принимать решение о вызове функции. Она чаще "не слышит" команду и сваливается в обычный ответ.

#### **Разрыв модальностей отсутствует**
- Accuracy_A (Text) = 94%.
- Accuracy_D (Audio) = 94%.  

Нет большой разницы между текстовым и голосовым вводом при использовании каскадной схемы.

#### **Краткий пром. итог**
Базовая модель Gemma-3n демонстрирует высокие способности к следованию инструкциям. Главная зона роста - снижение количества ложных срабатываний FP.


## ⚡ LoRA fine-tuning

- Дообучать на русский язык (напр. CommonVoice) большого смысла нет. Модель неплохо понимает русский язык.
- Дообучение ASR (pipeline B) тоже вероятно не даст большго прироста в качестве, поскольку модель уже неплохо справляется с преобразованием речи в текст. (если не считать грамматику и фонетику)
- Pipeline D (Cascaded B + A) показал себя лучше других. Он зависит от текстовой модели (pipeline A). Улучшим текстовую модель - улучшим и Pipeline D.


### LoRa

#### Конфигурация

In [ ]:
from trl import SFTTrainer #, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# на время обучения минизиурем промпт (ОТМЕНЕНО. используем полный промпт)
TRAIN_SYSTEM_PROMPT = """Ты — ИИ-ассистент.
Твоя задача: отвечать текстом (Чат) или генерировать XML <tool_call> (Инструмент).

ПРАВИЛА:
1. Отвечай на языке пользователя.
2. Формат: только чистый XML или текст. Без Markdown.
3. ЛОГИКА:
   - "Напиши/Исправь/Переведи/Код" -> <tool_call>...
   - "Объясни/Посоветуй/Привет/Вопрос" -> Текст.
"""


# настройка модели
model = FastModel.get_peft_model(
    model,
    r =  64, #64, # было 16,
    lora_alpha = 128, # было 16,
    lora_dropout = 0.0, # добавляем шум, чтобы модель не переобучалась

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],


    # ⚠️ НЕ АКТУАЛЬНО - ошибка OOM исправлена (gradient_checkpointing = True)
    # Обучение всех модулей (gate, up, down) в режиме FP32 - это ту мач для T4
    # Оставим только самые важные для внимания (q, v).
    # target_modules =  ["q_proj", "v_proj"],

    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = 2048,
    use_rslora = False,
    loftq_config = None,
)



# на инференсе используем полный промпт (SYSTEM_PROMPT_A, который дал Acc = 94%)
INFERENCE_SYSTEM_PROMPT = """Ты — ИИ-ассистент для редактирования текста.
Твоя задача: либо просто ответить пользователю (Чат), либо сгенерировать готовый XML для вставки в приложение (Инструмент).

### ГЛАВНЫЕ ПРАВИЛА
1. **ЯЗЫК:** Всегда отвечай на том же языке, на котором спрашивает пользователь.
2. **ФОРМАТ:** Никогда не используй Markdown (```xml или ```json). Выводи чистый текст или чистый XML в одну строку.
3. **РОЛЬ:** Ты — инструмент работы с текстом. Ты умеешь и писать с нуля (произвольный тест, стихи, код, письма), и редактировать. Если просят что-то создать — создавай. Не отказывайся.
4. **ДИСЦИПЛИНА:** ВСЕГДА следуй логике выбора режима ниже.

---

### ЛОГИКА ВЫБОРА РЕЖИМА

**РЕЖИМ 1: ИНСТРУМЕНТ (Вставка текста)**
Используй этот режим, если пользователь хочет **СОЗДАТЬ** или **ИЗМЕНИТЬ** текст/код.
Триггеры (Глаголы действия):
- **Напиши / Составь / Сгенерируй** ("Напиши письмо", "Составь список", "Составь чек-лист")
- **Придумай / Создай** ("Придумай название", "Создай 3 варианта")
- **Исправь / Перепиши / Сократи / Улучши** ("Исправь ошибки", "Сделай вежливее")
- **Переведи** ("Переведи на английский")
- **Код** ("Напиши функцию")

ФОРМАТ ОТВЕТА (Строго XML в одну строку без лишних слов):
<tool_call><name>magic_paste</name><arguments><text>ТВОЙ_ГОТОВЫЙ_ТЕКСТ</text></arguments></tool_call>

---

**РЕЖИМ 2: ЧАТ (Обычный разговор)**
Используй этот режим, если пользователь хочет **УЗНАТЬ** информацию, поблагодарить или просто поболтать.
Триггеры (Глаголы восприятия):
- **Расскажи / Объясни** ("Расскажи про...", "Объясни как работает...")
- **Вопросы** ("Что такое...", "Кто победил...", "Как переводится...")
- **Вежливость** ("Привет", "Спасибо", "Пока")
- **Посоветуй / Подскажи** ("Посоветуй фильм")
- **Как переводится** ("Как переводится на английский...")



ФОРМАТ ОТВЕТА:
Простой текст ответа. Кратко и по делу.

### ПРИМЕРЫ (Обучение)

Пользователь: "Привет, как дела?"
Ответ: Привет! Я готов к работе с текстом.
Пояснение: Вежливость -> Чат

Пользователь: "Придумай 3 идеи для стартапа."
Ответ: <tool_call><name>magic_paste</name><arguments><text>1. Умный сад\n2. AI-репетитор\n3. Доставка дронами</text></arguments></tool_call>
Пояснение: Команда "Придумай" -> Инструмент

Пользователь: "Напиши функцию сортировки на JS."
Ответ: <tool_call><name>magic_paste</name><arguments><text>const sort = (arr) => arr.sort();</text></arguments></tool_call>
Пояснение: Команда "Напиши" -> Инструмент

Пользователь: "Объясни, как работает сортировка."
Ответ: Сортировка упорядочивает элементы в массиве по определенному критерию, например, по возрастанию чисел.
Пояснение: Просьба "Объясни" -> Чат

Пользователь: "Перепиши это вежливо: Ты опоздал."
Ответ: <tool_call><name>magic_paste</name><arguments><text>К сожалению, вы задержались.</text></arguments></tool_call>
Пояснение: Команда "Перепиши" -> Инструмент

Пользователь: "Переведи на английский: Привет мир"
Ответ: <tool_call><name>magic_paste</name><arguments><text>Hello World</text></arguments></tool_call>
Пояснение: Команда "Переведи" -> Инструмент

Пользователь: "Как переводится: Привет мир"
Ответ:  "Привет мир переводится на английский как 'Hello World'."
Пояснение: Вопрос "Как переводится" -> Чат

### КОНЕЦ ПРИМЕРОВ. НАЧАЛО ДИАЛОГА:
"""

Unsloth: Making `model.base_model.model.model.language_model` require gradients


#### Данные

In [ ]:
def format_prompts(examples):
    texts = []

    for user, assistant in zip(examples['user_text'], examples['assistant_text']):

        # формируем диалог (используем краткий промпт для обучения)
        messages = [
            {"role": "system", "content": INFERENCE_SYSTEM_PROMPT }, # TRAIN_SYSTEM_PROMPT},
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ]

        # apply_chat_template превращает это в строку cо спецтокенами
        # <bos><start_of_turn>system...<end_of_turn>...
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)

    return { "text" : texts }


# применяем и чистим датасет от всего лишнего (аудио и прочего)
train_ds_clean = train_ds.map(format_prompts, batched=True)
# оставляем ТОЛЬКО колонку text
train_ds_clean = train_ds_clean.select_columns(["text"])

# проверим есть ли в конце токен конца генерации
print(train_ds_clean[0]['text'][-100:])

print("✅ Данные отформатированы. Пример:")
print(train_ds_clean[0]['text'][:200])



Map:   0%|          | 0/150 [00:00<?, ? examples/s]

я. Предлагаю согласовать другое время для обсуждения.</text>
</arguments>
</tool_call><end_of_turn>

✅ Данные отформатированы. Пример:
<bos><start_of_turn>user
Ты — ИИ-ассистент для редактирования текста.
Твоя задача: либо просто ответить пользователю (Чат), либо сгенерировать готовый XML для вставки в приложение (Инструмент).

### Г


#### Кастомный коллатор
- импорт DataCollatorForCompletionOnlyLM из trl выдает ошибку
- возможно версия trl не подходящая


In [ ]:
from transformers import DataCollatorForLanguageModeling
import numpy as np
text_tokenizer = tokenizer.tokenizer
class CustomMaskingCollator(DataCollatorForLanguageModeling):
    def __init__(self, tokenizer, resp_temp, mlm=False):
        super().__init__(tokenizer=tokenizer, mlm=mlm)
        # кодируем шаблон в токены один раз
        self.resp_temp_ids = tokenizer.encode(resp_temp, add_special_tokens=False)
        print(f"токена шаблона: {self.resp_temp_ids}")

    def torch_call(self, examples):
        # базовая обработка паддинг и т.д.
        batch = super().torch_call(examples)

        # проходим по каждому примеру
        for i in range(len(batch["labels"])):
            # ищем шаблон в input_ids
            input_ids = batch["input_ids"][i].tolist()

            # находим начало шаблона
            found_idx = -1
            n = len(self.resp_temp_ids)
            for j in range(len(input_ids) - n):
                if input_ids[j:j+n] == self.resp_temp_ids:
                    found_idx = j + n # индекс ПЕРВОГО токена ответа
                    break

            if found_idx != -1:
                # маскируем всё ДО ответа - инструкцию и вопрос
                # -100 = не считать лосс для этих токенов
                batch["labels"][i, :found_idx] = -100
            else:
                # если шаблон не найден — маскируем вообще всё
                print(f"⚠️ Шаблон не найден {i}")
                batch["labels"][i, :] = -100

        return batch

# gemma 3 обычно использует такой формат перед ответом
resp_temp = "<start_of_turn>model\n"
collator = CustomMaskingCollator(tokenizer=text_tokenizer, resp_temp=resp_temp)

токена шаблона: [105, 4368, 107]


#### trainer

In [ ]:
from trl import SFTTrainer, SFTConfig
max_seq_length = 2048

# конфиг
args = SFTConfig(
    output_dir = "outputs",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    max_steps = 60,    # лосс стабильно начинает расти после 20-го шага
    warmup_steps = 5,     # быстрый разогрев. было 10
    lr_scheduler_type = "cosine",  # плавное затухание
    learning_rate = 5e-5,

    save_strategy = "steps",     # сохраняем чекпоинты
    save_steps = 5,
    completion_only_loss=True,

    # ⚠️ РЕШЕНИЕ ПРОБЛЕМЫ Out Of Memory (VRAM)
    gradient_checkpointing = True,
    neftune_noise_alpha = 5,  # шумовая защита от переобучения

    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    remove_unused_columns = True,

    report_to = "none",
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 3407,
)


trainer = SFTTrainer(
    model = model,
    tokenizer = text_tokenizer,
    train_dataset = train_ds_clean,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = collator,

    args = args,

)




Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/150 [00:00<?, ? examples/s]

### 🚀 Запуск LoRa (In progress ...)

In [ ]:
torch.cuda.empty_cache()
gc.collect()

print("\n 🚀 Запуск обучения")
trainer_stats = trainer.train()

print("\n ✅ Обучение завершено!")
print(f"Время: {trainer_stats.metrics['train_runtime']:.2f} сек")

# инференс
FastModel.for_inference(model)

# используем те же тестовые данные
test_inputs = [item['user_text'] for item in test_ds]
test_targets = [item['assistant_text'] for item in test_ds]

# используем полный промпт
print('Проводим оценку с полным промптом')
print(f'INFERENCE_SYSTEM_PROMPT: {INFERENCE_SYSTEM_PROMPT[:120]} ...' )
lora_predictions_isp = run_pipeline_a_batched(
    test_inputs,
    model,
    tokenizer,
    batch_size=8,
    system_prompt=INFERENCE_SYSTEM_PROMPT
)

# оценка
metrics_lora_isp = evaluate_tool_pipeline(
    pipeline_name="Pipeline A (LoRa Tuned)",
    user_inputs=test_inputs,
    expected_outputs=test_targets,
    generated_outputs=lora_predictions_isp
)


# сохранение
model.save_pretrained("lora_model_magic_paste")
tokenizer.save_pretrained("lora_model_magic_paste")





 🚀 Запуск обучения


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 150 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 160,759,808 of 8,010,738,000 (2.01% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,7.480500
2,8.149000
3,4.099600
4,3.116600
5,2.605300
6,2.394600
7,2.131000
8,1.941800
9,1.590100
10,1.604900



 ✅ Обучение завершено!
Время: 2303.51 сек
Проводим оценку с полным промптом
INFERENCE_SYSTEM_PROMPT: Ты — ИИ-ассистент для редактирования текста.
Твоя задача: либо просто ответить пользователю (Чат), либо сгенерировать го ...
Добавляем 6 текстов для паддинга 
🚀 Запуск Pipeline A: 56 примеров (Batch size: 8)


100%|██████████| 7/7 [05:28<00:00, 46.95s/it]



📊 ЗАПУСК ОЦЕНКИ: Pipeline A (LoRa Tuned)
✅ Accuracy (Intent): 100.00% (Общая точность)
🎯 Precision:         100.00% (Насколько метко вызываем тулы)
📡 Recall:            100.00% (Сколько команд мы 'услышали')
------------------------------
🚨 False Alarm Rate:  0.00% (Ложные срабатывания)
📝 XML Syntax Score:  100.00% (Валидность XML)

🎉 Ошибок нет.


['lora_model_magic_paste/processor_config.json']

### 💾 Сохраняем

In [ ]:
# сохраняем адаптеры (LoRa) локально
output_dir = "lora_gemma_fixed"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Локально сохранено в {output_dir}")

# сохраняем на drive
from google.colab import drive
import os

if os.path.exists("/content/drive"):
    print("Drive уже примонтирован")
else:
    drive.mount('/content/drive')

# копируем папку с моделью на диск
dest_path = f"/content/drive/MyDrive/{output_dir}"
!cp -r {output_dir} "{dest_path}"

print(f"✅ Модель сохранена на Google Диске: {dest_path}")

# cкачиваем архивом
!zip -r lora_gemma_3n.zip {output_dir}
from google.colab import files
files.download('lora_gemma_3n.zip')

✅ Локально сохранено в lora_gemma_fixed
Mounted at /content/drive
✅ Модель сохранена на Google Диске: /content/drive/MyDrive/lora_gemma_fixed
  adding: lora_gemma_fixed/ (stored 0%)
  adding: lora_gemma_fixed/tokenizer.model (deflated 52%)
  adding: lora_gemma_fixed/README.md (deflated 65%)
  adding: lora_gemma_fixed/adapter_model.safetensors (deflated 11%)
  adding: lora_gemma_fixed/tokenizer_config.json (deflated 97%)
  adding: lora_gemma_fixed/special_tokens_map.json (deflated 77%)
  adding: lora_gemma_fixed/adapter_config.json (deflated 57%)
  adding: lora_gemma_fixed/processor_config.json (deflated 20%)
  adding: lora_gemma_fixed/preprocessor_config.json (deflated 55%)
  adding: lora_gemma_fixed/tokenizer.json (deflated 83%)
  adding: lora_gemma_fixed/chat_template.jinja (deflated 71%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Итоговые выводы



Было проведено исследование применимости мультимодальной модели Gemma-3n-4B для создания голосового ассистента, управляющего текстовым вводом.

#### **Сравнение архитектур**
- Разделение задач ASR и LLM в Pipeline D оказалось эффективнее чем подход End-to-End в Pipeline C.
- Pipeline D показал результаты, идентичные чисто текстовой задаче (p. A). Качество ASR (p. B) достаточно высоко. Модель работает надежно, разделяя задачи восприятия и мышления.
- Pipeline C показал падение Recall до 83%. Модели сложнее одновременно обрабатывать аудиопоток и принимать решение о вызове функции.

#### **Результаты Fine-Tuning**
Базовая модель (p. A) показала высокую точность ~94%, но страдала от ложных срабатываний (False Alarm Rate ~11.5%). Она пыталась выполнить команду (вызвать инструмент) даже на развлекательные запросы или информационные запросы.

После применения LoRA на специально подготовленном датасете удалось достичь хороших показателей на (небольшой) тестовой выборке :
- Accuracy: 94% → 100%
- False Alarm Rate: 11.5% → 0.00%

Fine-tuning позволил сформировать четкую границу принятия решений. Модель научилась хорошо отличать намерения "Напиши/Измени" (действие) от "Расскажи/Объясни" (информация/чат).

#### **Программная реализация**
Работа доведена до стадии MVP приложения, работающего в условиях ограниченных ресурсов:
- Выполнена конвертация весов в формат GGUF с решением проблемы OOM на T4 (Google Colab) из-за требований FP32.
- Проведено квантование до Q4_K_M, позволившее уместить модель и контекст в память видеокарты RTX 3050 4 GB (laptop).
- Разработан унифицированный сервис на базе паттернов Strategy/Factory для замены провайдеров ASR (Whisper/Gemma) и TTS (Edge/Silero).
- Реализован UI без перехвата системного фокуса на базе PySide6 с кастомной отрисовкой и анимациями состояния.
- Внедрено управление процессами через Windows Job Objects, гарантирующее корректное завершение C++ сервера и освобождение VRAM при сбоях.
- Реализовано низкоуровневое управление буфером обмена и эмуляция ввода для устранения коллизий фокуса при вставке текста.


---



.

.

.


---
---
---
## Конвертация в .GGUF формат.
- неудачные попытки сконвертировать модель в .GGUF в среде Google Colab
- оставлено исключительно для истории
- в конченом счете процесс конвертации был перенесен на локальный ПК (после  
стадии слияния модели с LoRa адаптерами), из-за невозможности конвертации на T4

In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import drive
import os


saved_model_path = "lora_gemma_fixed"

print(f"Загружаем модель из: {saved_model_path} ...")

# загружаем модель и токенизатор
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = saved_model_path,
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = torch.float16,
    device_map = "auto",

)

# переводим в режим инференса
FastLanguageModel.for_inference(model)

print("✅ Модель загружена")


import gc

# очистка памяти перед тяжелой операцией
torch.cuda.empty_cache()
gc.collect()

print("⏳ Начинаем конвертацию в GGUF...")

# конвертация и сохранение (Q4_K_M)
model.save_pretrained_gguf(
    "gemma_3n_magic_paste_q4_k_m",
    tokenizer,
    quantization_method = "q4_k_m"
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Загружаем модель из: lora_gemma_fixed ...
==((====))==  Unsloth 2026.2.1: Fast Gemma3N patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3n won't work! Using float32.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Модель загружена


Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


⏳ Начинаем конвертацию в GGUF...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Preparing safetensor model files:  75%|███████▌  | 3/4 [06:29<02:23, 143.93s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [09:21<00:00, 140.36s/it]


Unsloth: Merge process complete. Saved to `/content/gemma_3n_magic_paste_q4_k_m`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: Failed to convert text model to GGUF: Command 'python llama.cpp/unsloth_convert_hf_to_gguf.py --outfile gemma-3n-e4b-it.F16.gguf --outtype f16 --split-max-size 50G gemma_3n_magic_paste_q4_k_m' returned non-zero exit status 137.

### ⚠️ Веса сохранились, но процесс конвертации завершился ошибкой...  
- заново запускаем блок кода "установка библиотек"
- загружаем сохраненную копию весов с гугл диска
- СНАЧАЛА делаем слиянием LoRa адаптеров с чистой моделью, чтобы минимизировать потребление памяти на этапе конвертации в GGUF
- потом конвертируем в GGUF и квантуем


Swap

In [ ]:
# # монтируем диск
# from google.colab import drive
# drive.mount('/content/drive')

# # создаем SWAP 8GB чтобы избежать вылета Colab по памяти
# !fallocate -l 8G /swapfile
# !chmod 600 /swapfile
# !mkswap /swapfile
# !swapon /swapfile
# print("✅ Swap file created and enabled (8GB)")


# # монтируем диск
# from google.colab import drive
# drive.mount('/content/drive')

# # создаем SWAP 8GB через dd (медленнее fallocate, но работает всегда)
# !dd if=/dev/zero of=/swapfile bs=1G count=8 status=progress
# !chmod 600 /swapfile
# !mkswap /swapfile
# !swapon /swapfile
# print("✅ Swap file created and enabled (8GB) via dd")


# монтируем диск
# from google.colab import drive
# drive.mount('/content/drive')

# # создаем SWAP 8GB в Google Drive (ext4 поддерживает swap)
# !dd if=/dev/zero of=/content/drive/MyDrive/swapfile bs=1G count=8 status=progress
# !chmod 600 /content/drive/MyDrive/swapfile
# !mkswap /content/drive/MyDrive/swapfile
# !swapon /content/drive/MyDrive/swapfile
# print("✅ Swap file created and enabled (8GB) in Google Drive")

Mounted at /content/drive
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 28 s, 156 MB/s
4+0 records in
4+0 records out
4294967296 bytes (4.3 GB, 4.0 GiB) copied, 27.5198 s, 156 MB/s
^C
Setting up swapspace version 1, size = 4 GiB (4294963200 bytes)
no label, UUID=b9795fa8-adde-41d9-834b-e74456a91815
swapon: /content/drive/MyDrive/swapfile: swapon failed: Invalid argument
✅ Swap file created and enabled (8GB) in Google Drive


In [ ]:
# монтируем диск
from google.colab import drive
import os
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# создаем SWAP 8GB через dd (медленно, но надежно)
if not os.path.exists('/swapfile'):
    print("Создаем Swap-файл")
    !dd if=/dev/zero of=/swapfile bs=1G count=8 status=progress
    !chmod 600 /swapfile
    !mkswap /swapfile
    !swapon /swapfile
    print("✅ Swap file created and enabled (8GB)")
else:
    print("✅ Swap file already exists")

# проверка памяти
!free -h




✅ Swap file already exists
               total        used        free      shared  buff/cache   available
Mem:            12Gi       994Mi       2.3Gi       4.0Mi       9.4Gi        11Gi
Swap:             0B          0B          0B


In [ ]:
%%capture
!pip install unsloth
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install "huggingface_hub>=0.24.0"
!pip install "transformers>=4.43.0"

In [ ]:
!free -h
!swapon --show

               total        used        free      shared  buff/cache   available
Mem:            12Gi       780Mi       9.1Gi       5.0Mi       2.8Gi        11Gi
Swap:             0B          0B          0B


#### Слияние LoRa и базовой модели

In [ ]:
import torch
from unsloth import FastLanguageModel
import gc

# путь к адаптерам на диске
adapter_path = "/content/drive/MyDrive/lora_gemma_fixed"
output_merged_path = "/content/merged_model_hf"

print(f"Загрузка адаптеров из: {adapter_path}")

# загружаем модель
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# сливаем веса и выгружаем LoRa
print("Идет merge")
model.save_pretrained_merged(
    output_merged_path,
    tokenizer,
    save_method = "merged_16bit",
)
print(f"✅ Слияние завершено. Модель сохранена в: {output_merged_path}")

# чистим память
del model
del tokenizer
torch.cuda.empty_cache()
gc.collect()



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Загрузка адаптеров из: /content/drive/MyDrive/lora_gemma_fixed
==((====))==  Unsloth 2026.2.1: Fast Gemma3N patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3n won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.15G [00:00<?, ?B/s]

...

Unsloth: Preparing safetensor model files:  75%|███████▌  | 3/4 [05:17<01:46, 106.51s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [14:02<00:00, 210.73s/it]


Unsloth: Merge process complete. Saved to `/content/merged_model_hf`
✅ Слияние завершено. Модель сохранена в: /content/merged_model_hf


92263

Скопируем на google drive, чтобы не терять время на merge в след. раз.

In [ ]:
!cp -r /content/merged_model_hf /content/drive/MyDrive/merged_model_hf

Конвертация и Квантование

In [ ]:
!pip install numpy==1.26.4  # фиксит numpy conflicts


In [ ]:
import os

# клонируем и собираем llama.cpp
# if not os.path.exists("llama.cpp"):
#     !git clone https://github.com/ggerganov/llama.cpp
#     !cd llama.cpp && make clean && make -j 4
# Удаляем старую папку и клонируем заново
%cd /content
!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# Устанавливаем CMake зависимости
!apt update -qq && apt install -qq cmake build-essential -y

# Собираем с CUDA для Colab T4
!cmake -B build -DGGML_CUDA=ON
!cmake --build build --config Release -j2

# Проверяем бинарники
!ls -la build/bin/

# Фиксим окружение
import os
os.environ["PYTHONPATH"] = "/content/llama.cpp:" + os.environ.get("PYTHONPATH", "")

# # устанавливаем зависимости
# !pip install -r llama.cpp/requirements.txt

/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 80064, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 80064 (delta 1), reused 1 (delta 1), pack-reused 80054 (from 2)
Receiving objects: 100% (80064/80064), 294.68 MiB | 17.34 MiB/s, done.
Resolving deltas: 100% (57887/57887), done.
Updating files: 100% (2272/2272), done.
/content/llama.cpp
48 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
build-essential is already the newest version (12.9ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info


In [ ]:

# правильная установка
!pip install -q numpy==1.26.4 transformers torch huggingface_hub

# клонируем llama.cpp (только конвертер!)
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# устанавливаем Python зависимости для конвертера
!pip install -r requirements.txt

# конвертация HF → F16 GGUF (ЕДИНСТВЕННЫЙ РАБОЧИЙ СПОСОБ)
!python convert_hf_to_gguf.py /content/merged_model_hf \
    --outfile /content/temp_f16.gguf \
    --outtype f16

# cPU-only сборка (3 минуты вместо 37!)
!cmake -B build
!cmake --build build --config Release -j2

# квантование
!./build/bin/llama-quantize /content/temp_f16.gguf /content/gemma_3n_q4_k_m.gguf Q4_K_M

# проверка
!ls -lh /content/gemma_3n_q4_k_m.gguf

!cp -r /content/gemma_3n_q4_k_m.gguf /content/drive/MyDrive/gemma_3n_q4_k_m.gguf


Cloning into 'llama.cpp'...
remote: Enumerating objects: 80064, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
^C
[Errno 2] No such file or directory: 'llama.cpp'
/content/llama.cpp/llama.cpp
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
INFO:hf-to-gguf:Loading model: merged_model_hf
INFO:hf-to-gguf:Model architecture: Gemma3nForConditionalGeneration
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing 

In [ ]:
# 1. ПРОВЕРЬТЕ что есть
!ls -la /content/merged_model_hf/
!ls -la /content/llama.cpp/build/bin/llama-quantize

# 2. Доконвертируйте (5-10 минут, не прерывайте!)
%cd /content/llama.cpp
!python convert_hf_to_gguf.py /content/merged_model_hf \
    --outfile /content/temp_f16.gguf \
    --outtype f16

# 3. Квантуйте (2-3 минуты)
!./build/bin/llama-quantize /content/temp_f16.gguf /content/gemma_3n_q4_k_m.gguf Q4_K_M

# 4. Проверьте результат
!ls -lh /content/gemma_3n_q4_k_m.gguf


total 15370852
drwxr-xr-x 3 root root       4096 Feb 14 19:44 .
drwxr-xr-x 1 root root       4096 Feb 14 20:21 ..
drwxr-xr-x 3 root root       4096 Feb 14 19:38 .cache
-rw-r--r-- 1 root root       1626 Feb 14 19:38 chat_template.jinja
-rw-r--r-- 1 root root       5675 Feb 14 19:38 config.json
-rw-r--r-- 1 root root 3077103824 Feb 14 19:45 model-00001-of-00004.safetensors
-rw-r--r-- 1 root root 4966792808 Feb 14 19:55 model-00002-of-00004.safetensors
-rw-r--r-- 1 root root 4992870216 Feb 14 19:57 model-00003-of-00004.safetensors
-rw-r--r-- 1 root root 2663414864 Feb 14 19:58 model-00004-of-00004.safetensors
-rw-r--r-- 1 root root     171459 Feb 14 19:38 model.safetensors.index.json
-rw-r--r-- 1 root root        777 Feb 14 19:38 special_tokens_map.json
-rw-r--r-- 1 root root    1204203 Feb 14 19:38 tokenizer_config.json
-rw-r--r-- 1 root root   33442807 Feb 14 19:38 tokenizer.json
-rw-r--r-- 1 root root    4696020 Feb 14 19:38 tokenizer.model
ls: cannot access '/content/llama.cpp/build/b

In [ ]:
# конвертация HF -> GGUF
# используем сохраненную на прошлом шаге папку merged_model_hf

!python llama.cpp/convert_hf_to_gguf.py /content/merged_model_hf \
    --outfile /content/temp_f16.gguf \
    --outtype f16

print("\n ✅ Конвертация в  GGUF завершена")

# квантование до Q4_K_M
if os.path.exists("/content/temp_f16.gguf"):

    !./llama.cpp/llama-quantize /content/temp_f16.gguf /content/gemma_3n_q4_k_m.gguf q4_k_m

    print("\n✅ Квантование завершено!")

    # удаляем временный файл
    !rm /content/temp_f16.gguf
else:
    print("❌ Ошибка: файл FP16 не был создан.")

# сохранение результата на Google Drive
if os.path.exists("/content/gemma_3n_q4_k_m.gguf"):
    dest_path = "/content/drive/MyDrive/gemma_3n_q4_k_m.gguf"
    !cp /content/gemma_3n_q4_k_m.gguf "{dest_path}"
    print(f"🎉 Готово! Файл сохранен на Google drive: {dest_path}")
else:
    print("❌ Финальный файл не найден.")

python3: can't open file '/content/llama.cpp/llama.cpp/convert_hf_to_gguf.py': [Errno 2] No such file or directory

 ✅ Конвертация в  GGUF завершена
❌ Ошибка: файл FP16 не был создан.
❌ Финальный файл не найден.


In [ ]:
# 1. НАЙДИТЕ llama-quantize (правильный путь)
!find /content/llama.cpp -name "llama-quantize" -type f
!ls -la /content/llama.cpp/bin/llama-quantize || echo "НЕТ в bin/"
!ls -la /content/llama.cpp/llama.cpp/build/bin/llama-quantize || echo "НЕТ в llama.cpp/build/bin/"


/content/llama.cpp/llama.cpp/build/bin/llama-quantize
ls: cannot access '/content/llama.cpp/bin/llama-quantize': No such file or directory
НЕТ в bin/
-rwxr-xr-x 1 root root 442512 Feb 14 21:26 /content/llama.cpp/llama.cpp/build/bin/llama-quantize


In [ ]:
#  Конвертация
%cd /content/llama.cpp
!python convert_hf_to_gguf.py /content/merged_model_hf \
    --outfile /content/temp_f16.gguf \
    --outtype f16


/content/llama.cpp
INFO:hf-to-gguf:Loading model: merged_model_hf
INFO:hf-to-gguf:Model architecture: Gemma3nForConditionalGeneration
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00004-of-00004.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:altup_proj.weight,                 torch.bfloat16 --> F16, shape = {2048, 2048, 3}
INFO:hf-to-gguf:altup_unembd_proj.weight,          torch.bfloat16 --> F16, shape = {2048, 2048, 3}
INFO:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> F16, shape = {2048, 262400}
INFO:hf-to-gguf:Padding per-layer embeddings shape [262144, 8960] from 2621

In [ ]:
from google.colab import files

# Скачиваем ВСЕ файлы
files_list = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

for filename in files_list:
    full_path = f'/content/merged_model_hf/{filename}'
    print(f"Скачиваю: {filename}")
    files.download(full_path)


Скачиваю: model-00001-of-00004.safetensors


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Локально
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
cmake -B build -DGGML_CUDA=1
cmake --build build --config Release
python convert_hf_to_gguf.py merged_model_hf --outfile gemma_lora_f16.gguf --outtype f16
./build/bin/llama-quantize gemma_lora_f16.gguf gemma_lora_q4_k_m.gguf Q4_K_M


In [ ]:
import time

for x in range(1000_000_000):
    print(x)
    time.sleep(10)

0
1
2
3
4
5
6
7
8
9
10
11
12


KeyboardInterrupt: 